In [1]:
"""
https://pypi.org/project/mysql-connector-python/
https://dev.mysql.com/doc/dev/connector-python/latest/installation.html
"""
print("Referencia")

Referencia


In [2]:
#!pip install -r requirements.txt

In [3]:
import os
import pandas as pd
from tqdm import tqdm
import mysql.connector

from mysql.connector import Error
from dotenv import load_dotenv

In [4]:
load_dotenv('.env')

True

In [5]:
archivoParquet = os.getenv("RutaArchivoParquet")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")
database = os.getenv("database")
table = os.getenv("table")

### Test Basico

In [6]:
# Conectarse al servidor
cnx = mysql.connector.connect(
    host=host,
    port=port,
    user=user,
    password=password)

# Obtener un cursor
cur = cnx.cursor()

# Ejecutar una consulta
cur.execute("SELECT CURDATE()")

# Obtener un resultado
row = cur.fetchone()
print("Current date is: {0}".format(row[0]))

# Cerrar conexión
cnx.close()

Current date is: 2025-03-13


In [7]:
def leer_parquet(ruta_archivo):
    try:
        # Leer el archivo Parquet usando pandas
        df = pd.read_parquet(ruta_archivo, engine='pyarrow')
        print("Archivo Parquet leído exitosamente.")
        return df
    except Exception as e:
        print(f"Error al leer el archivo Parquet: {e}")
        return None

In [8]:
def conectar_mysql(host, user, port, password, database):
    try:
        conexion = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )
        if conexion.is_connected():
            print("Conexión exitosa a MySQL.")
            return conexion
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
        return None

In [9]:
def insertar_datos(conexion, tabla, datos):  
    try:  
        cursor = conexion.cursor()  
          
        # Crear una consulta SQL para insertar datos  
        columnas = ", ".join(datos.columns)  
        placeholders = ", ".join(["%s"] * len(datos.columns))  
        sql = f"INSERT INTO {tabla} ({columnas}) VALUES ({placeholders})"  
          
        # Convertir el DataFrame a una lista de tuplas  
        valores = [tuple(row) for row in datos.to_numpy()]  
          
        # Ejecutar la consulta en lote con una barra de progreso  
        for i in tqdm(range(0, len(valores), 1000), desc="Insertando datos"):  
            batch = valores[i:i + 1000]  
            cursor.executemany(sql, batch)  
            conexion.commit()  
          
        print(f"{cursor.rowcount} filas insertadas exitosamente.")  
    except Error as e:  
        print(f"Error al insertar datos: {e}")  
    finally:  
        cursor.close() 

In [10]:
data = leer_parquet(archivoParquet)
data.shape

Archivo Parquet leído exitosamente.


(105397, 16)

In [11]:
if data is not None:
    # Conectar a MySQL
    conexion = conectar_mysql(host, user, port, password, database)
    if conexion:
        # Insertar datos en MySQL
        insertar_datos(conexion, table, data)
        # Cerrar la conexión
        conexion.close()

Conexión exitosa a MySQL.


Insertando datos: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:44<00:00,  2.38it/s]

397 filas insertadas exitosamente.


### Tener en cuenta

Primero se debe crear la base de datos, posterior la tabla basado en el tipo campos del parquet, 